In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_file
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, ColorBar, HoverTool
from bokeh.core.properties import value

In [2]:
career_progress_file = 'data.csv'
dashboard_file = 'index.html'

In [3]:
timeseries_df = pd.read_csv(career_progress_file)

def get_cumulative_sum(df, obtain_from, set_to):
    df[set_to] = df.get(obtain_from).cumsum()
    return df

bat_list = ['Root', 'Roy', 'Bairstow', 'Buttler', 'Morgan', 'Stokes']
colors = ['blue', 'purple', 'green', 'black', 'red', 'violet']

for batsman in bat_list:
    temp_tot = batsman + '_tot'
    timeseries_df = get_cumulative_sum(timeseries_df, batsman, temp_tot)

In [5]:
line_cds = ColumnDataSource(timeseries_df)

select_tools = ['box_select', 'box_zoom', 'lasso_select', 'tap', 'reset']

fig_lday = figure(
    title='Career Runs Progression',
    x_axis_label='Innings',
    y_axis_label='Runs',
    x_minor_ticks=5,
    y_range=(0, 7000),
    tools=select_tools,
    toolbar_location='right',
    plot_width=1900,
    plot_height=800
    )

def add_line_to_figure(figure, x_axis, batsman, colors):
    figure.line(
    x = x_axis,
    y = batsman+'_tot', 
    source = line_cds, 
    color = colors,
    line_width = 3,
    legend = value(batsman)
    )


def add_circle_to_figure(figure, x_axis, batsman):
    figure.circle(
    x=x_axis,
    y=batsman+'_tot',
    source=line_cds,
    color='black',
    selection_color='darkgreen',
    nonselection_color='lightgray',
    size=5.2,
    alpha=0.5
    )
    

def add_glyph_to_figure(figure, x_axis, batsman):
    hover_glyph = figure.circle(
        x=x_axis,
        y=batsman+'_tot',
        source=line_cds,
        hover_fill_color='black',
        hover_alpha=0.5,
        size=15,
        alpha=0,
        )
    tooltips = [('Innings', '@'+x_axis), ('Match Runs', '@'+batsman), ('Cumulative Runs', '@'+batsman+'_tot')]
    figure.add_tools(HoverTool(tooltips=tooltips,
                   renderers=[hover_glyph]))

for batsman, color in zip(bat_list, colors):
    add_line_to_figure(fig_lday, 'Innings', batsman, color)
    add_circle_to_figure(fig_lday, 'Innings', batsman)
    add_glyph_to_figure(fig_lday, 'Innings', batsman)

fig_lday.legend.location = "top_left"
fig_lday.legend.click_policy="hide"

output_file(dashboard_file)
show(fig_lday)